# Stock Market Analysis

## Business Problem

# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pandas_datareader
import pandas_datareader.data as web
import datetime
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tools.eval_measures import rmse
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')

# Functions

In [2]:
def get_data(stock):
    start = datetime.datetime(2011,1,1)
    end = datetime.datetime(2021,6,1)
    stock_df = web.DataReader(stock, 'yahoo', start, end)
    stock_df['Stock'] = stock
    return stock_df

# Load Data

In [3]:
#tesla = web.DataReader('TSLA', 'yahoo', start, end)

### Top 10 Tech Stocks

In [4]:
apple = get_data('AAPL')

In [5]:
microsoft = get_data('MSFT')

In [6]:
google = get_data('GOOG')

In [7]:
amazon = get_data('AMZN')

In [8]:
facebook = get_data('FB')

In [9]:
tencent = get_data('TCEHY')

In [10]:
alibaba = get_data('BABA')

In [11]:
tesla = get_data('TSLA')

In [12]:
tsmc = get_data('TSM')

In [13]:
samsung = get_data('005930.KS')

In [26]:
to_concat = [apple, microsoft, google, amazon, facebook, tencent, alibaba, tesla, tsmc, samsung]
tech_df = pd.concat(to_concat)

In [27]:
tech_df.shape

(24859, 7)

In [28]:
aapl_df = tech_df[tech_df['Stock'] == 'AAPL']
aapl_df.shape

(2620, 7)

In [30]:
aapl_df.iloc[-1]

High              125.35
Low               123.94
Open              125.08
Close             124.28
Volume       6.76371e+07
Adj Close         124.28
Stock               AAPL
Name: 2021-06-01 00:00:00, dtype: object

In [29]:
msft_df = tech_df[tech_df['Stock'] == 'MSFT']
msft_df.shape

(2620, 7)

### Top 10 Crypto

In [18]:
bitcoin = get_data('BTC-USD')

In [19]:
bitcoin.head()

,High,Low,Open,Close,Volume,Adj Close,Stock
Date,,,,,,,
2014-09-16,468.174011,452.421997,465.864014,457.334015,21056800.0,457.334015,BTC-USD
2014-09-17,456.859985,413.104004,456.859985,424.440002,34483200.0,424.440002,BTC-USD
2014-09-18,427.834991,384.532013,424.102997,394.795990,37919700.0,394.795990,BTC-USD
2014-09-19,423.295990,389.882996,394.673004,408.903992,36863600.0,408.903992,BTC-USD
2014-09-20,412.425995,393.181000,408.084991,398.821014,26580100.0,398.821014,BTC-USD


### Top 10

# Data Exploration

# Modeling